# Premissa

Este desafio visa aplicar os modelos construídos anteriormente em um novo mapa com a finalidade de encontrar um caminho mais curto.

## Desvendando a Problemática

### Mapa do Desafio
![alternative text](./mapa_desafio.png)

### Distância em Linha Reta
![alternative text](./distancia_linha_reta.png)

1) Estado inicial: cidade de Porto União;
2) Estado final: cidade de Curitiba;
3) Espaço de estados: todas as possíveis caminhos entre Porto União e Curitiba, ex.:
> * Porto União >> São Mateus >> Lapa >> Contenda >> Araucária >> Curitiba;
> * Porto União >> Canoinhas >> Mafra >> Tijucas do Sul >> São José dos Pinhais >> Curitiba;
> * Porto União >> Paulo Frontin >> Irati >> Palmeira >> Campo Largo >> Balsa Nova >> Curitiba;
4) Ações de passagem: ir de uma cidade até outra, ex.:
> * Porto União >> São Mateus;
> * Palmeira >> Campo Largo;
> * Lapa >> Contenda;
5) Solução: processamento dos caminhos buscando encontrar a distância mais curta;


# Imports

In [1]:
import json
from typing import List

import numpy as np
from module import Mapa, CidadeAdjacente

# Vértice

Para o desenvolvimento deste desafio, será mantido a classe de Vértice construída nos arquivos anteriores;

In [2]:
class Vertice:
    '''
    Classe responsável por criar as cidades
    '''
    def __init__(self, rotulo, distancia_objetivo):
        self.rotulo = rotulo
        self.distancia_objetivo = distancia_objetivo
        self.visitado = False
        self.adjacentes = []

    def adiciona_adjacentes(self, adjacente):
        self.adjacentes.append(adjacente)

    def mostra_adjacentes(self):
        for i in self.adjacentes:
            print(f'{i.vertice.rotulo}: {i.custo}')

# Adjacente

O mesmo se aplica a classe adjacente que será reaproveitada dos arquivos anteriores, no caso desta classe optou-se por fazer uso da versão implementada no arquivo aestrela, visto que esta é a versão mais completa

In [3]:
class Adjacente:
    '''
    Classe responsável por fazer a conexão entre as cidades
    '''
    def __init__(self, vertice, custo):
        self.vertice = vertice
        self.custo = custo
        self.distancia_real = self.vertice.distancia_objetivo + self.custo

# Vetor Ordenado

Por fim o Vetor Ordenado também será reutilizado por estar em bom estado de reaproveitamento.

In [19]:
class VetorOrdenado:
    '''
    Classe responsável por criar um vetor de ordenação de valores focado na distancia_real das adjacentes
    '''
    def __init__(self, capacidade):
        self.capacidade = capacidade
        self.ultima_posicao = -1
        self.lista_adjacentes = np.empty(self.capacidade, dtype=object)

    def adicionar_valor_vetor(self, adjacente):
        if self.ultima_posicao == self.capacidade - 1:
            print('Capacidade máxima alcançada.')
        else:
            posicao = 0
            for i in range(self.ultima_posicao + 1):
                posicao = i
                if self.lista_adjacentes[i].distancia_real > adjacente.distancia_real:
                    break
                if i == self.ultima_posicao:
                    posicao = i + 1

            x = self.ultima_posicao
            while x >= posicao:
                self.lista_adjacentes[x + 1] = self.lista_adjacentes[x]
                x -= 1

            self.lista_adjacentes[posicao] = adjacente
            self.ultima_posicao += 1

# Grafo

Para a classe da cosntrução do Grafo, será implementado um modelo de criação automática que irá ler um arquivo json e converter ele no Grafo do desagio.

## Desenvolvimento de automação para criação de Grafo

In [31]:
# Chamando json do mapa
with open('mapa_regiao_curitiba.json', encoding='utf-8') as file:
    mapa_regiao_curitiba: List[Mapa] = json.load(file)

for cidade in mapa_regiao_curitiba:
    print(cidade)
    print('')

{'nome': 'Porto União', 'distancia_linha_reta': 203, 'adjacentes': [{'nome': 'Paulo Frontin', 'distancia_real': 46}, {'nome': 'São Mateus', 'distancia_real': 87}, {'nome': 'Canoinhas', 'distancia_real': 78}]}

{'nome': 'Paulo Frontin', 'distancia_linha_reta': 172, 'adjacentes': [{'nome': 'Irati', 'distancia_real': 75}, {'nome': 'Porto União', 'distancia_real': 46}]}

{'nome': 'Canoinhas', 'distancia_linha_reta': 141, 'adjacentes': [{'nome': 'Porto União', 'distancia_real': 78}, {'nome': 'Três Barras', 'distancia_real': 12}, {'nome': 'Mafra', 'distancia_real': 66}]}

{'nome': 'Três Barras', 'distancia_linha_reta': 131, 'adjacentes': [{'nome': 'São Mateus', 'distancia_real': 43}, {'nome': 'Canoinhas', 'distancia_real': 12}]}

{'nome': 'São Mateus', 'distancia_linha_reta': 123, 'adjacentes': [{'nome': 'Irati', 'distancia_real': 57}, {'nome': 'Palmeira', 'distancia_real': 77}, {'nome': 'Lapa', 'distancia_real': 60}, {'nome': 'Três Barras', 'distancia_real': 43}, {'nome': 'Porto União', 'di

In [6]:
# Definindo classe construtor
class Grafo:
    def __init__(self, mapa: List[Mapa]):
        self.cidades = self.criar_vertices_grafo(mapa)

    def adicionar_adjacente_grafo(self, cidade: CidadeAdjacente, vertice: Vertice, grafo: List[Vertice]):
            vertice.adiciona_adjacentes(
                Adjacente(
                    next(x for x in grafo if x.rotulo == cidade['nome']),
                    cidade['distancia_real']
                )
            )
            return vertice

    def encontrar_cidade_adjacente(self, grafo: List[Vertice], mapa: List[Mapa]):
        for vertice in grafo:
            cidade = next(x for x in mapa if x['nome'] == vertice.rotulo)
            for adjacente in cidade['adjacentes']:
                vertice = self.adicionar_adjacente_grafo(
                    adjacente, 
                    vertice, 
                    grafo
                )
        return grafo

    def criar_vertices_grafo(self, mapa: List[Mapa]):
        grafo: List[Vertice] = []
        for cidade in mapa:
            vertice = Vertice(cidade['nome'], cidade['distancia_linha_reta'])
            grafo.append(vertice)
        return self.encontrar_cidade_adjacente(grafo, mapa)

## Teste de criação de Grafo

In [26]:
def mostraDetalhes(cidade: Vertice):
    print('--------------------------------------------------')
    print(f'Cidade: {cidade.rotulo}')
    
    cidade.adjacentes.sort(key=lambda x: x.distancia_real, reverse=False)
    if cidade.distancia_objetivo > 0:
        print(f'Distância Linha Reta: {cidade.distancia_objetivo}')
        print('Adjacentes || Dist Real || Dist Linha Reta || Soma(Peso)')
        for idx in range(len(cidade.adjacentes)):
            print('{}º - {} || {} || {} || {}'.format(
                idx+1,
                cidade.adjacentes[idx].vertice.rotulo,
                cidade.adjacentes[idx].custo,
                cidade.adjacentes[idx].vertice.distancia_objetivo,
                cidade.adjacentes[idx].distancia_real
            ))

In [27]:
#Testando funcionamento das classe de criação de grafo
grafo = Grafo(mapa_regiao_curitiba)
for item in grafo.cidades:
    mostraDetalhes(item)

# Busca AEstrela

In [17]:
# Construção de classe Gulosa
class AEstrela:
    def __init__(self, objetivo: str):
        self.objetivo = objetivo
        self.encontrado = False

    def buscar(self, atual: Vertice):
        mostraDetalhes(atual)
        atual.visitado = True
        
        if atual == self.objetivo:
            self.encontrado = True
        else:
            vetor = VetorOrdenado(len(atual.adjacentes))
            for adjacente in atual.adjacentes:
                if not adjacente.vertice.visitado:
                    adjacente.vertice.visitado = True
                    vetor.adicionar_valor_vetor(adjacente)
                    
            if vetor.lista_adjacentes[0] != None:
                self.buscar(vetor.lista_adjacentes[0].vertice)

In [28]:
#Testando modelo de Busca Gulosa saindo Porto União até Curitiba
buscador = AEstrela(grafo.cidades[-1])
buscador.buscar(grafo.cidades[0])

--------------------------------------------------
Cidade: Porto União
Distância Linha Reta: 203
Adjacentes || Dist Real || Dist Linha Reta || Soma(Peso)
1º - São Mateus || 87 || 123 || 210
2º - Paulo Frontin || 46 || 172 || 218
3º - Canoinhas || 78 || 141 || 219
--------------------------------------------------
Cidade: São Mateus
Distância Linha Reta: 123
Adjacentes || Dist Real || Dist Linha Reta || Soma(Peso)
1º - Lapa || 60 || 74 || 134
2º - Palmeira || 77 || 59 || 136
3º - Três Barras || 43 || 131 || 174
4º - Irati || 57 || 139 || 196
5º - Porto União || 87 || 203 || 290
--------------------------------------------------
Cidade: Lapa
Distância Linha Reta: 74
Adjacentes || Dist Real || Dist Linha Reta || Soma(Peso)
1º - Contenda || 26 || 39 || 65
2º - Mafra || 57 || 94 || 151
3º - São Mateus || 60 || 123 || 183
--------------------------------------------------
Cidade: Contenda
Distância Linha Reta: 39
Adjacentes || Dist Real || Dist Linha Reta || Soma(Peso)
1º - Araucária || 18 |